In [5]:
import os
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    # Directory where the base model and other related files are stored
    root_dir: Path
    
    # Path to the pre-trained base model file
    base_model_path: Path
    
    # Path where the updated base model will be saved after modifications
    updated_base_model_path: Path
    
    # List defining the size of input images (e.g., [224, 224, 3])
    params_image_size: list
    
    # Learning rate used for training the model (e.g., 0.01)
    params_learning_rate: float
    
    # Boolean flag indicating whether to include the top layers of the model (True/False)
    params_include_top: bool
    
    # String specifying the weights initialization method (e.g., 'imagenet')
    params_weights: str
    
    # Number of output classes for the classification task (e.g., 2 for binary classification)
    params_classes: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager class.

        Args:
            config_filepath (Path): Path to the YAML configuration file.
            params_filepath (Path): Path to the YAML parameters file.
        """
        # Load the configuration and parameters from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the directory specified in the configuration file for artifacts
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Retrieves the configuration for preparing the base model.

        Returns:
            PrepareBaseModelConfig: An instance of PrepareBaseModelConfig populated with data from the configuration and parameters files.
        """
        # Extract the 'prepare_base_model' section from the configuration file
        config = self.config.prepare_base_model
        
        # Create the directory specified for the base model
        create_directories([config.root_dir])

        # Instantiate and return the PrepareBaseModelConfig with values from the configuration and parameters
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2024-08-07 22:35:37.185735: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-07 22:35:39.062576: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-07 22:35:39.063828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 22:35:43.194320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        """
        Initializes the PrepareBaseModel class with the provided configuration.

        Args:
            config (PrepareBaseModelConfig): Configuration for preparing the base model.
        """
        self.config = config

    def get_base_model(self):
        """
        Loads the VGG16 model with specified configurations and saves it.

        Actions:
            - Loads the VGG16 model with specified image size, weights, and top layer inclusion.
            - Saves the model to the path specified in the configuration.
        """
        # Load the VGG16 model using specified parameters
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Save the loaded base model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepares the full model by adding a classification layer on top of the base model.

        Args:
            model (tf.keras.Model): The base model to be extended.
            classes (int): Number of output classes.
            freeze_all (bool): Whether to freeze all layers of the base model.
            freeze_till (int): Number of layers from the end of the model to be unfrozen.
            learning_rate (float): Learning rate for the optimizer.

        Returns:
            tf.keras.Model: The full model with a classification head.
        """
        # Freeze layers based on the freeze_all and freeze_till parameters
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        # Add a new classification layer on top of the base model
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        # Create a new model with the updated top layer
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        # Compile the model
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        # Print the model summary
        full_model.summary()
        return full_model

    def update_base_model(self):
        """
        Updates the base model by preparing it for classification and saving the updated model.

        Actions:
            - Prepares the full model by adding a classification head.
            - Saves the updated model to the path specified in the configuration.
        """
        # Prepare the full model with the classification head
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        # Save the updated model
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the model to the specified path.

        Args:
            path (Path): Path to save the model.
            model (tf.keras.Model): Model to be saved.
        """
        model.save(path)

In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2024-08-07 22:38:35,133: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-08-07 22:38:35,138: INFO: common: YAML file: params.yaml loaded successfully]
[2024-08-07 22:38:35,140: INFO: common: Created directory at: artifacts]
[2024-08-07 22:38:35,141: INFO: common: Created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 1s 0us/step
[2024-08-07 22:38:37,132: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                 